駅間ルートをShapeファイルからGeoJSONに変換  
加えて、駅間ルートにsub_idを紐付ける

In [73]:
import geopandas as gpd
import pandas as pd
from shapely import Point, equals_exact
from queue import Queue
import csv
from collections import defaultdict

from app.utils.db import get_db_adapter
from app.utils.logger import getLogger

logger = getLogger(__name__)
db_adapter = get_db_adapter()

In [74]:
agency = "関東自動車"

In [75]:
shp = gpd.read_file(f"./data/駅間ルート（手動）/{agency}/駅間ルート.geojson")
stops = gpd.read_file(f"./data/駅間ルート（手動）/{agency}/stops.geojson")
# stop_times = pd.read_csv(f"./data/GTFS/{agency}/stop_times.txt")

In [76]:
del shp["Id"]
shp["sub_id"] = range(1, len(shp) + 1)
shp.to_file(f"./data/駅間ルート（手動）/{agency}/駅間ルート_.geojson", driver="GeoJSON")
single_shp = shp.explode(index_parts=False)
dup_shp = single_shp[single_shp.index.duplicated(keep=False)]
dup_shp.to_file(
    f"./data/駅間ルート（手動）/{agency}/駅間ルート_確認用.geojson", driver="GeoJSON"
)

/Users/shibuki/Documents/GTFS_delay/.venv/lib/python3.11/site-packages/geopandas/io/file.py:572: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)


もし、次のコードでエラーが出るなら、MultiLineStringだからです。"] ], ["でgeojsonを検索して、強引にLineStringに変換しましょう

In [77]:
if len(dup_shp) > 0:
    raise Exception
shp = single_shp.copy()
shp.head()

,sub_id,geometry
0,1,"LINESTRING (139.88529 36.62298, 139.88582 36.6..."
1,2,"LINESTRING (139.88820 36.62088, 139.88836 36.6..."
2,3,"LINESTRING (139.88854 36.61228, 139.88875 36.6..."
3,4,"LINESTRING (139.88854 36.61228, 139.88825 36.6..."
4,5,"LINESTRING (139.89113 36.60842, 139.89103 36.6..."


In [78]:
shp["O"] = shp.geometry.apply(lambda k: Point(k.coords[0]))
shp["D"] = shp.geometry.apply(lambda k: Point(k.coords[-1]))
shp.head()

/Users/shibuki/Documents/GTFS_delay/.venv/lib/python3.11/site-packages/geopandas/geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)
/Users/shibuki/Documents/GTFS_delay/.venv/lib/python3.11/site-packages/geopandas/geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)


,sub_id,geometry,O,D
0,1,"LINESTRING (139.88529 36.62298, 139.88582 36.6...",POINT (139.88529 36.62298),POINT (139.88820 36.62088)
1,2,"LINESTRING (139.88820 36.62088, 139.88836 36.6...",POINT (139.88820 36.62088),POINT (139.88920 36.61593)
2,3,"LINESTRING (139.88854 36.61228, 139.88875 36.6...",POINT (139.88854 36.61228),POINT (139.88920 36.61593)
3,4,"LINESTRING (139.88854 36.61228, 139.88825 36.6...",POINT (139.88854 36.61228),POINT (139.89113 36.60842)
4,5,"LINESTRING (139.89113 36.60842, 139.89103 36.6...",POINT (139.89113 36.60842),POINT (139.88538 36.60533)


In [79]:
# Point、駅名、IDが格納されたリストを作成
stops_info_list = stops[["geometry", "stop_name", "stop_id"]].to_numpy().tolist()
# 中間地点のID、一つのポイントにつき一意に割り当てる
intermediate_point_id = 1

座標から駅名、IDを割り当てるコード  
toleranceは許容範囲。もし、エラーが発生するようなら、許容値を小さくしましょう。

In [80]:
def lookup_stops(x: Point) -> pd.Series:
    global intermediate_point_id
    l = []
    for geo, name, id in stops_info_list:
        if equals_exact(x, geo, tolerance=0.0000001):
            l.append([name, id])

    if len(l) == 0:
        # 該当する駅名が存在しない場合は、中間地点のIDを割り当てる
        l.append([f"IP_{intermediate_point_id}", f"IP_{intermediate_point_id}"])
        stops_info_list.append([x, *l[0]])
        intermediate_point_id += 1

    if len(l) == 1:
        return pd.Series(l[0])

    raise ValueError(f"候補が複数存在します。\n{l}")

In [81]:
shp[["O_Name", "O_Id"]] = shp["O"].apply(lookup_stops)
shp[["D_Name", "D_Id"]] = shp["D"].apply(lookup_stops)
shp

/Users/shibuki/Documents/GTFS_delay/.venv/lib/python3.11/site-packages/geopandas/geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)
/Users/shibuki/Documents/GTFS_delay/.venv/lib/python3.11/site-packages/geopandas/geoseries.py:645: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)
/Users/shibuki/Documents/GTFS_delay/.venv/lib/python3.11/site-packages/geopandas/geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, 

,sub_id,geometry,O,D,O_Name,O_Id,D_Name,D_Id
0,1,"LINESTRING (139.88529 36.62298, 139.88582 36.6...",POINT (139.88529 36.62298),POINT (139.88820 36.62088),ニュー富士見,S0904000494,ニュー富士見上,S0904000493
1,2,"LINESTRING (139.88820 36.62088, 139.88836 36.6...",POINT (139.88820 36.62088),POINT (139.88920 36.61593),ニュー富士見上,S0904000493,ニュー富士見中,S0904000492
2,3,"LINESTRING (139.88854 36.61228, 139.88875 36.6...",POINT (139.88854 36.61228),POINT (139.88920 36.61593),ニュー富士見団地入口,S0904000491,ニュー富士見中,S0904000492
3,4,"LINESTRING (139.88854 36.61228, 139.88825 36.6...",POINT (139.88854 36.61228),POINT (139.89113 36.60842),ニュー富士見団地入口,S0904000491,瓦谷町東,S0904000490
4,5,"LINESTRING (139.89113 36.60842, 139.89103 36.6...",POINT (139.89113 36.60842),POINT (139.88538 36.60533),瓦谷町東,S0904000490,豊郷台中央公園前,S0904000489
...,...,...,...,...,...,...,...,...
1104,1105,"LINESTRING (139.86018 36.57412, 139.86028 36.5...",POINT (139.86018 36.57412),POINT (139.86621 36.56985),IP_109,IP_109,IP_45,IP_45
1105,1106,"LINESTRING (139.85822 36.56402, 139.85800 36.5...",POINT (139.85822 36.56402),POINT (139.85407 36.56425),三の沢,S0904000212,西三の沢,S0904000213
1106,1107,"LINESTRING (139.85407 36.56425, 139.85380 36.5...",POINT (139.85407 36.56425),POINT (139.85048 36.56358),西三の沢,S0904000213,IP_110,IP_110
1107,1108,"LINESTRING (139.85048 36.56358, 139.85046 36.5...",POINT (139.85048 36.56358),POINT (139.84935 36.56360),IP_110,IP_110,IP_115,IP_115


端の駅名、ID付きのGISデータの出力

In [82]:
del shp["O"]
del shp["D"]

In [83]:
shp.to_file(f"./data/駅間ルート/{agency}/駅間ルート.geojson", driver="GeoJSON")

辺情報付き無向グラフを用いて、駅間のIDをまとめる。

In [84]:
# 駅ID：[[繋がっている駅ID、sub_id],...]
nodes = defaultdict(list)
# グラフの探索可能距離
MAX_DEPTH = 2

# グラフの初期化
for _, x in shp.iterrows():
    nodes[x["O_Id"]].append([x["D_Id"], x["sub_id"]])
    nodes[x["D_Id"]].append([x["O_Id"], x["sub_id"]])

res = []

for o in nodes.keys():
    # 中間地点は除外
    if o.startswith("IP_"):
        continue

    visited = []
    q = Queue()

    q.put([o, [], 0])

    while not q.empty():
        v, sub_ids, depth = q.get()
        visited.append(v)

        for node, sub_id in nodes[v]:
            # すでに訪れたことのある駅の場合には除外
            if node in visited:
                continue

            if node.startswith("IP_"):
                # 中間地点は続行
                q.put([node, [*sub_ids, sub_id], depth])
            else:
                # 探索閾値以内であれば、結果を保存して続行
                if (depth + 1) < MAX_DEPTH:
                    q.put([node, [*sub_ids, sub_id], depth + 1])
                # 結果を保存
                res.append([o, node, *sub_ids, sub_id])

# 書き込み
with open(f"./data/駅間ルート/{agency}/駅間_subid.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["O", "D"])
    writer.writerows(res)

駅間の組み合わせで、漏れがないかを確認

In [85]:
OD_list = [[*x[:2]] for x in res]
exception_list = []
for _, g in stop_times.groupby("trip_id"):
    O = ""
    for _, x in g.iterrows():
        if (
            O != ""
            and [O, x["stop_id"]] not in OD_list
            and [O, x["stop_id"]] not in exception_list
        ):
            exception_list.append([O, x["stop_id"]])
        O = x["stop_id"]
print(exception_list)

NameError: name 'stop_times' is not defined